# Doc2Vecで４人の小説家の小説をベクトル化後、kmeansでクラスタリング

## 初期設定

In [0]:
# Mecabや諸々インストール
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab

# 追加辞書インストール
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n

# 辞書を mecab-ipadic-NEologdに変更
#「/etc/mecabrc」ファイルの「dicdir = /var/lib/mecab/dic/debian」を、
#「mecab-ipadic-NEologd」の辞書のパス「/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd」に変更する。
!sed -i -e "s!/var/lib/mecab/dic/debian!/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd!g" /etc/mecabrc

# swingインストール
!apt-get -q -y install swig

# mecab-python3をインストール
!pip install mecab-python3

# gensimのインストール
!pip install gensim

# 日本語フォントのインストール
!apt-get -y install fonts-ipafont-gothic

## Pythonライブラリのインストール

In [0]:
import itertools
import random
import MeCab
from urllib import request 
import re
import numpy as np
import pandas as pd
from gensim import corpora, models # models.doc2vec.TaggedDocument() で、Doc2Vecが読める形式に変換する
from gensim.test.utils import get_tmpfile
from sklearn.cluster import KMeans
from scipy.cluster.vq import vq, whiten
# from collections import defaultdict
import pickle

## 学習データの読み込み  
梶井基次郎: 45作品  
永井荷風:   85作品  
中島敦:     27作品  
夏目漱石:   104作品

In [65]:
# google driveよりファイル取り出し
file_name = '/content/drive/My Drive/novel_list.pickle'
pickle_dic = open(file_name, 'rb')
novel_df = pickle.load(pickle_dic)
pickle_dic.close

<function BufferedReader.close>

In [66]:
novel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261 entries, 0 to 260
Data columns (total 3 columns):
title      261 non-null object
content    261 non-null object
author     261 non-null object
dtypes: object(3)
memory usage: 6.2+ KB


In [0]:
# モデリング結果との連結キーとしてID列追加
id_list = []
for i in range(len(novel_df)):
  ID = 'ID_' + str(i)
  id_list.append(ID)

novel_df['ID'] = id_list

In [68]:
novel_df.head(5)

,title,content,author,ID
0,筧の話,\n\n 私は散歩に出るのに二つの路を持っていた。一つは渓《たに》に沿った街道で、もう一つは...,0,ID_0
1,『戰旗』『文藝戰線』七月號創作評,\n\n 『戰旗』\n\n\n 彼女等の會話 （窪川いね子氏）\n\n この月讀...,0,ID_1
2,不幸,\n\n［＃５字下げ］第二稿［＃「第二稿」は中見出し］\n\n 師走のある寒い夜のことである...,0,ID_2
3,編輯後記（大正十五年四月號）,\n 忽那が三人寄せ書きの後記を書かうと云つて、よしとは云つたもののこれと云つて書く程のこと...,0,ID_3
4,泥濘,\n\n［＃５字下げ］一［＃「一」は中見出し］\n\n それはある日の事だった。――\n 待...,0,ID_4


In [69]:
# データをランダムに入れ替え
novel_df = novel_df.sample(frac=1, random_state=0)
novel_df.head(5)

,title,content,author,ID
166,教育と文芸,\n\n 私は思いがけなく前から当地の教育会の御招待を受けました。凡《およ》そ一カ月前に御通...,3,ID_166
106,十九の秋,\n\n 近年新聞紙の報道するところについて見るに、東亜の風雲はますます急となり、日支同文の...,1,ID_106
239,作物の批評,\n\n 中学には中学の課目があり、高等学校には高等学校の課目があって、これを修了せねば卒業...,3,ID_239
76,放水路,\n\n 隅田川《すみだがわ》の両岸は、千住《せんじゅ》から永代《えいたい》の橋畔《きょうは...,1,ID_76
220,京に着ける夕,\n\n 汽車は流星の疾《はや》きに、二百里の春を貫《つらぬ》いて、行くわれを七条《しちじょ...,3,ID_220


## 対象データの前処理とTokenizerの用意  
参考サイト：https://qiita.com/dcm_sawayama/items/406408e8bda0840a8106  
参考サイト：https://qiita.com/Hironsan/items/2466fe0f344115aff177  
#### *   英単語はすべて小文字にする
#### *   全角カタカタ、半角カタカナの表記ゆれ（週刊誌サイトなのでないと思う・・・）  
#### *   全角数字を半角数字に変換  
#### *   全角記号：（）とか、―とか、「」、？などの削除（mecabで記号未選択にするだけ。）  
#### *   数値の削除  
#### *   日本語のストップワード削除  
#### *   英語のストップワード削除  
#### *   WordNetによる単語の統一（一旦は行わずにモデル作ってみる） 
#### *   Tokunizerの用意 

In [0]:
# 日本語ストップワードのダウンロード
res = request.urlopen("http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt")
stopwords = [line.decode("utf-8").strip() for line in res]

# 英語ストップワードのダウンロード
res = request.urlopen("http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/English.txt")
stopwords += [ line.decode("utf-8").strip() for line in res]

In [0]:
# 分かち書きして、対象単語のみ取り出すTokenizerクラスの作成
# 処理内容
#   英単語を小文字に変える
#   分かち書きする
#   名詞、動詞、形容詞のみ抽出
#   ストップワードを除外する
#   ※上記処理後の結果を見て追加処理考える

class Tokenizer:

    def __init__(self, stopwords, parser=None, include_pos=None, exclude_posdetail=None, exclude_reg=None):
        '''
        stopwords: ストップワードのリスト
        parser: mecabがデフォルト。別のパーサーを指定したときに利用
        include_pos: 抽出対象の品詞リスト。デフォルトは、['名詞', '動詞', '形容詞']
        exclude_posdetail: 除外対象の詳細品詞("固有名詞"とか、"格助詞"など)。デフォルトは、['接尾', '数']
        exclude_reg: 除外対象文字を正規表現で指定できる。例) r"\d(年|月|日) 指定で年月日を削除できる。
        '''
        self.stopwords = stopwords
        self.include_pos = include_pos if include_pos else ['名詞', '動詞', '形容詞']
        self.exclude_posdetail = exclude_posdetail if exclude_posdetail else ['接尾', '数']
        self.exclude_reg = exclude_reg if exclude_reg else r"$^"

        if parser:
            self.parser = parser
        else:
            # MeCab.Tager("-Ochasen")指定すると、分かち書き結果の出力形式が -Ochasen 無しと異なるので注意
            mecab = MeCab.Tagger('-Ochasen -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')
            self.parser = mecab.parse

    
    def tokenize(self, text, show_pos=False):
        '''
        - 引数の記事本文から、対象単語のリストを抽出して返す。
        text: 一つの記事
        show_pos: 単語の品詞も同時に表示するかの指定。
            False: 単語のみのリスト
            True: (単語, 品詞)のタプルのリストになる。
        '''
        text = text.lower() # 英字の小文字化
        text = re.sub(r"([0-9０-９])", "", text) # 半角、全角数字の削除

        # 文章をMeCab.parseでパースしたのを改行(\n)でパースして一行毎にし、それをさらにタブ(\t)でsplitしてリスト化する。
        # l の中身：textが、'メカブって神ってる' の場合
        # [
        #    ['メカブ', 'メカブ', 'MeCab', '名詞-固有名詞-一般', '', ''],
        #    ['って', 'ッテ', 'って', '助詞-格助詞-連語', '', ''],
        #    ['神ってる', 'カミッテル', '神ってる', '名詞-固有名詞-一般', '', ''],
        #    ['EOS'],
        #    ['']
        # ]
        l = [line.split("\t") for line in self.parser(text).split("\n")]

        # lから一行(単語一つのパース結果のリスト)ずつ取り出し、
        # そのリストの要素数が4つ以上なら、品詞のある単語なので、抽出チェックする
        # i ：単語一つのパース結果のリスト：['神ってる', 'カミッテル', '神ってる', '名詞-固有名詞-一般', '', '']
        # i[2] : 単語自身
        # i[3] : 単語の品詞： '名詞-固有名詞-一般'
        # i[3].split("-")[0] in self.include_pos : 単語の品詞が対象かのチェック
        # i[3].split("-")[1] not in self.exclude_posdetail : 単語の詳細品詞が、除外対象かチェック
        # not re.search(r"(-|−)\d", i[2]) : 単語が -数字, −数字 なら除外
        # not re.search(self.exclude_reg, i[2]) : 単語が除外正規表現に当てはまるかのチェック
        # i[2] not in self.stopwords : 単語がストップワードかのチェック
        # res 中身例１（show_pos=Falseの時)：['MeCab', 'って', '神ってる']
        # res 中身例２（show_pos=Trueの時)：[('MeCab', '名詞-固有名詞-一般'), ('って', '助詞-格助詞-連語'), ('神ってる', '名詞-固有名詞-一般')]        
        res = [
            i[2] if not show_pos else (i[2], i[3]) for i in l
                if len(i) >= 4 # 品詞があるとき
                    and i[3].split("-")[0] in self.include_pos
                    and i[3].split("-")[1] not in self.exclude_posdetail
                    and not re.search(r"(-|−)\d", i[2])
                    and not re.search(self.exclude_reg, i[2])
                    and i[2] not in self.stopwords
        ] 
        return res

In [73]:
# Tokunizerの起動確認用に元データ表示
test_text = novel_df.iloc[0,1]
test_text

'\n\n\u3000私は思いがけなく前から当地の教育会の御招待を受けました。凡《およ》そ一カ月前に御通知がありましたが、私は、その時になって見なければ、出られるか出られぬか分らぬために、直《すぐ》にお答をすることが出来ませんでした。しかし、御懇切《ごこんせつ》の御招待ですから義理にもと思いまして体だけ出｜懸《か》けて参りました。別に面白いお話も出来ません、前《ぜん》申した通り体だけ義理にもと出かけたわけであります。\n\u3000私のやる演題はこういう教育会の会場での経験がないのでこまりました。が、名が教育会であるし、引受ける私は文学に関係あるものであるから、教育と文芸という事にするが能《よ》いと思いまして、こういう題にしました。この教育と文芸というのは、諸君が主であるからまげて教育をさきとしたのであります。\n\u3000よく誤解される事がありますので、そんな事があっては済みませんから、ちょっと注意を申述《もうしの》べて置きます。教育といえばおもに学校教育であるように思われますが、今私の教育というのは社会教育｜及《および》家庭教育までも含んだものであります。\n\u3000また私のここにいわゆる文芸は文学である、日本における文学といえば先《まず》小説｜戯曲《ぎきょく》であると思います。順序は矛盾しましたが、広義の教育、殊に、徳育とそれから文学の方面殊に、小説戯曲との関係連絡の状態についてお話致します。日本における教育を昔と今とに区別して相《あい》比較するに、昔の教育は、一種の理想を立て、その理想を是非実現しようとする教育である。しこうして、その理想なるものが、忠とか孝とかいう、一種抽象した概念を直《ただ》ちに実際として、即ち、この世にあり得るものとして、それを理想とさせた、即ち孔子を本家《ほんけ》として、全然その通りにならなくともとにかくそれを目あてとして行くのであります。\n\u3000なお委《くわ》しくいいますと聖人といえば孔子、仏《ほとけ》といえば釈迦《しゃか》、節婦《せっぷ》貞女忠臣孝子は、一種の理想の固《かた》まりで、世の中にあり得ないほどの、理想を以て進まねばならなかった。親が、子供のいう事を聞かぬ時は、二十四孝《にじゅうしこう》を引き出して子供を戒《いまし》めると、子供は閉口《へいこう》するというような風であります。それで昔は上の方には束縛がなくて

In [74]:
# Tokenizer()インスタンスを作成し、動くか確認
t = Tokenizer(stopwords + ["…。", "ーー", "＃", "字下げ", "は中見出し", "「", "」", "一", "［", "］"])
test_kekka = t.tokenize(test_text)
test_kekka[:10]

['思いがけない', '当地', '教育', '招待', '受ける', '凡', 'そ', '通知', 'ある', 'なる']

## モデリングと評価  


### 1.   全記事をTokenizer.tokenize()メソッドでパースして、docsリストに入れる  
### 2.   頻度の少ない単語（頻度の多い単語）を削除する  
###      　　？？？filter_extremes()の代わりは？？？  
#####     参考サイト：http://tadaoyamaoka.hatenablog.com/entry/2017/04/29/122128
### 3.   分かち書きした各小説を TaggedDocument()形式に変換する
### 4.   TaggedDocument の配列を使い Doc2Vec の学習モデルを作成し、モデルを保存する  
### 5.   kmeans で、クラスタリングを行う（クラスタ数は４）  
### 6.   クラスタ結果を集計する

### 1.   全小説をTokenizer.tokenize()メソッドでパースして、docsリストに入れる

In [76]:
# Tokenizer()インスタンスを作成
# ここで、追加のストップワードを追加。（適時調整）
t = Tokenizer(stopwords + ["…。", "ーー", "＃", "字下げ", "は中見出し", "「", "」", "一", "［", "］"])


# 各小説に対応する'ID'をID_listに入れる
# ※キーとなるIDは、文字列じゃないと後で参照できなくなるので注意

docs_list = []
ID_list = []
for i, record in novel_df.iterrows():
    docs_list.append(t.tokenize(record['content'])) # contentのパース
    ID_list.append(record[('ID')])

# 確認
print("docs_list: ", docs_list[0]) # 中身は「261行×各小説のパース結果」の２次元リスト
print("ID_list: ", ID_list[:10])

docs_list:  ['思いがけない', '当地', '教育', '招待', '受ける', '凡', 'そ', '通知', 'ある', 'なる', '見る', '出る', '出る', '分る', '直', 'お答え', 'する', '出来る', '御懇切', '懇切', '招待', '義理', '思う', '出', '懸る', '参る', '面白い', 'お話', '出来る', 'ぜん', '申す', '通り', '義理', '出かける', 'やる', '演題', '教育', '会場', '経験', 'ない', 'こまる', '教育', '引受ける', '文学', 'ある', '教育', '文芸', 'する', '能', '思う', '題', 'する', '教育', '文芸', 'の', '諸君', '主', 'まげる', '教育', 'さき', 'する', 'の', '誤解', 'する', 'ある', 'ある', '済む', '注意', '申', '述', 'もうす', 'べて', '置く', '教育', 'いう', '重荷', '学校教育', '思う', '教育', 'の', '社会教育', '及', 'お呼び', '家庭教育', '含む', 'ここに', '文芸', '文学', '日本', '文学', 'いう', '小説', '戯曲', '戯曲', '思う', '順序', '矛盾', 'する', '広義', '教育', '徳育', '文学', '方面', '殊', '小説', '戯曲', '連絡', '状態', 'お話', '致す', '日本', '教育', '昔', '区別', 'する', 'あい', '比較', 'する', '昔', '教育', '一種', '理想', '立てる', '理想', '是非', '実現', 'する', 'する', '教育', 'しこう', 'する', '理想', 'なる', '忠', '孝', '一種', '抽象', 'する', '概念', '直', 'ただ', 'ちる', 'する', '即値', 'この世', 'ある', '得る', '理想', 'する', '孔子', '本家', '本家', '通り', 'なる', '目あて', '行く', 'の', 'くう', 'する', 'くう', 'いる', '聖人', 'いう', 

In [77]:
novel_df[novel_df['ID']=='ID_166']['content']

166    \n\n　私は思いがけなく前から当地の教育会の御招待を受けました。凡《およ》そ一カ月前に御通...
Name: content, dtype: object

### 2. 頻度の少ない（多い）単語の削除：filter_extream()の代わり  
####---> 学習モデル作成時に引数で設定

### 3. 分かち書きした各小説を TaggedDocument形式に変換する

In [0]:
documents = [] # TaggedDocument形式にした小説データ保存用リスト

# TaggedDocument形式にして、配列に追加
for ID, article in zip(ID_list, docs_list):
    document = models.doc2vec.TaggedDocument(article, [ID])
    documents.append(document)

In [79]:
# 確認
documents[0]

TaggedDocument(words=['思いがけない', '当地', '教育', '招待', '受ける', '凡', 'そ', '通知', 'ある', 'なる', '見る', '出る', '出る', '分る', '直', 'お答え', 'する', '出来る', '御懇切', '懇切', '招待', '義理', '思う', '出', '懸る', '参る', '面白い', 'お話', '出来る', 'ぜん', '申す', '通り', '義理', '出かける', 'やる', '演題', '教育', '会場', '経験', 'ない', 'こまる', '教育', '引受ける', '文学', 'ある', '教育', '文芸', 'する', '能', '思う', '題', 'する', '教育', '文芸', 'の', '諸君', '主', 'まげる', '教育', 'さき', 'する', 'の', '誤解', 'する', 'ある', 'ある', '済む', '注意', '申', '述', 'もうす', 'べて', '置く', '教育', 'いう', '重荷', '学校教育', '思う', '教育', 'の', '社会教育', '及', 'お呼び', '家庭教育', '含む', 'ここに', '文芸', '文学', '日本', '文学', 'いう', '小説', '戯曲', '戯曲', '思う', '順序', '矛盾', 'する', '広義', '教育', '徳育', '文学', '方面', '殊', '小説', '戯曲', '連絡', '状態', 'お話', '致す', '日本', '教育', '昔', '区別', 'する', 'あい', '比較', 'する', '昔', '教育', '一種', '理想', '立てる', '理想', '是非', '実現', 'する', 'する', '教育', 'しこう', 'する', '理想', 'なる', '忠', '孝', '一種', '抽象', 'する', '概念', '直', 'ただ', 'ちる', 'する', '即値', 'この世', 'ある', '得る', '理想', 'する', '孔子', '本家', '本家', '通り', 'なる', '目あて', '行く', 'の', 'くう', 'する', 'くう', 'いる', '聖人

### 4. TaggedDocument の配列を使い Doc2Vec の学習モデルを作成し、モデルを保存する  
##### 参考サイト：https://yag-ays.github.io/project/pretrained_doc2vec_wikipedia/
##### 参考サイト：https://medium.com/@mishra.thedeepak/doc2vec-simple-implementation-example-df2afbbfbad5

In [0]:
# Doc2Vecの学習モデルの引数について
# model = models.Doc2Vec(documents, dm=1, vector_size=vec_size, window=10, alpha=alpha, min_alpha=0.00025, min_count=1, sample=0)
# 引数：
#   dm: アルゴリズム 1=dmpw, 0=DBOW
#   vector_size: 特徴ベクトルの次元数。（一つの小説を表現するベクトル数）
#   alpha: 学習率。低いほど精度が高いが、収束が遅くなる。高いほど収束が早いが、高すぎると発散する。
#   window: 学習する単語の前後数。
#   sample: あまりに高い頻度で出現する単語は意味のない単語の可能性が高いので無視する。その無視するときの閾値。
#   min_count: sampleとは逆に、頻度が少なすぎる単語もその文章を表現するのに適切でない場合もあるので無視する。その時の出現回数。1で一回でも出てきたのは無視しない。

In [0]:
# モデル用のパラメータ設定
# パラメータありすぎてどう設定すればいいのかわからないので、とりあえず学習回数のみ変化させて、他は適当に固定
MAX_EPOCHS = [10,20,30,40] # 学習回数
VEC_SIZE = 300 # 特徴ベクトルの次元数
ALPHA = 0.025
WINDOW = 10

In [0]:
# epochsを変化させて、それぞれのモデルを作成
model_list = []
for epochs in MAX_EPOCHS:
    model = models.Doc2Vec(documents, dm=1, vector_size=VEC_SIZE, window=WINDOW, alpha=ALPHA, min_count=1, sample=0, epochs=epochs, dbow_words=0)
    # modelの保存
    model_list.append(model)

In [82]:
# modelの精度検証
# 似た文章の抽出で、自分と同じ文章との類似確率が最も高いモデルを選ぶ
for epoch_no, model in zip(MAX_EPOCHS,model_list):
    print('epoch_no: ', epoch_no, '小説ID: ', ID_list[0])
    vector = model.infer_vector(docs_list[0])
    # model.docvecs.most_similar()で、引数の文書に最もコサイン類似度が近い文書から順に表示される。
    print('類似小説: ', model.docvecs.most_similar([vector], topn=981)[:3])

# 結果：とりあえず、epoch数10が一番類似度が高かったので、epoch数を10ので進める。

epoch_no:  10 小説ID:  ID_166
類似小説:  [('ID_166', 0.9768456220626831), ('ID_258', 0.9725338220596313), ('ID_177', 0.9648981094360352)]
epoch_no:  20 小説ID:  ID_166
類似小説:  [('ID_166', 0.9465136528015137), ('ID_194', 0.8768737316131592), ('ID_248', 0.8707756400108337)]
epoch_no:  30 小説ID:  ID_166


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


類似小説:  [('ID_166', 0.9447305798530579), ('ID_169', 0.7735674381256104), ('ID_248', 0.7536240816116333)]
epoch_no:  40 小説ID:  ID_166
類似小説:  [('ID_166', 0.9445764422416687), ('ID_169', 0.7087059020996094), ('ID_179', 0.6382896304130554)]


In [83]:
# 学習モデルの保存
fname = get_tmpfile('/content/drive/My Drive/d2v_model')
model.save

<bound method BaseAny2VecModel.save of <gensim.models.doc2vec.Doc2Vec object at 0x7f4bdff67ba8>>

In [62]:
# 備考１

# similarity(ID1, ID2) で、引数のID1, ID2 のコサイン類似度が返ってくる
# model.docvecs.similarity(2252, 2252)
model.docvecs.similarity('ID_1', 'ID_100')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


-0.015613741

In [63]:
# 備考２

# model.docvecs.vectors_docs で、各小説のベクトル表現を取得できる 
model.docvecs.vectors_docs

array([[ 0.8115828 ,  0.94194686,  2.8713589 , ...,  0.7550663 ,
        -0.62495506, -0.08764766],
       [-0.604434  , -0.20068459,  1.007455  , ..., -2.9897316 ,
         2.043359  ,  2.2638848 ],
       [ 2.3430128 ,  0.48355153,  1.6237613 , ..., -2.3480725 ,
         1.8805609 ,  2.3228617 ],
       ...,
       [ 1.3558177 ,  5.7981887 ,  2.5089343 , ..., -2.0962799 ,
         1.9182082 ,  5.670956  ],
       [ 0.38609365,  1.5527714 ,  0.26156124, ..., -0.1411277 ,
         0.07543117,  0.4401881 ],
       [-0.55202013,  0.6727353 ,  1.6992879 , ..., -1.5979153 ,
         1.7156782 , -0.7596251 ]], dtype=float32)

In [87]:
# 備考３

# docvec()で、id を指定すると、その小説のベクトル表現が手に入る
print('ベクトル表現：',model.docvecs['ID_0'][:20])
print('次元数：',len(model.docvecs['ID_0']))

ベクトル表現： [ 0.48478818 -0.9653436  -2.2068021  -0.25546056 -1.0126425   0.8391172
  0.23920082  0.99794996 -0.8177922  -0.29973534  1.1854297  -1.0960206
 -1.2742198  -0.7248557  -0.17067896  0.36536625 -0.9427723  -1.8624411
 -1.4816406   1.0113541 ]
次元数： 300


### 5. kmeans で、クラスタリングを行う 
##### 問題点：そもそも、各小説家の小説数が大きく異なっているのでkmeansでのクラスタリング自体に問題はないか？

In [0]:
# 白色化して文章ベクトルを単位超球面にする
whitened_vector = whiten(model.docvecs.vectors_docs)

# kmeans
kmeans_model = KMeans(n_clusters=4, init='k-means++', max_iter=300, random_state=1) # オブジェクト作成
pred = kmeans_model.fit_predict(whitened_vector) # クラスタ番号のリスト取得

In [0]:
# 結果のdf化
kmeans_df = pd.DataFrame({"ID": ID_list, "cluster": pred})
joined_df = pd.merge(kmeans_df, novel_df, how='inner', on=['ID'])
d2v_out_df = joined_df[['title','author','ID','cluster']]

In [102]:
d2v_out_df.head(3)

,title,author,ID,cluster
0,教育と文芸,3,ID_166,0
1,十九の秋,1,ID_106,1
2,作物の批評,3,ID_239,0


### 6. クラスタ結果を集計する  
#### 小説家ごとのクラスタ帰属状況をクロス集計する

In [0]:
# 作者名追加
no_to_name = {0:'梶井基次郎', 1:'永井荷風', 2:'中島敦', 3:'夏目漱石'}
d2v_out_df['auth_name'] = d2v_out_df['author'].map(no_to_name)

# 作者名でクロス集計
cross_d2v = pd.crosstab(d2v_out_df.auth_name, d2v_out_df.cluster)

In [106]:
cross_d2v

# ↓夏目漱石、永井荷風はとりあえずわけられている

cluster,0,1,2,3
auth_name,,,,
中島敦,1,0,17,9
夏目漱石,93,0,1,10
梶井基次郎,2,0,24,19
永井荷風,1,59,0,25


In [0]:
cross_d2v.to_excel('/content/drive/My Drive/d2v_kekka.xlsx')